In [1]:
import networkx as nx
import random
import heapq
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
import pandas as pd

In [2]:
def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5


def plot_graph_v2(G):
    pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

    # Lấy thuộc tính xác suất và trọng số của các cạnh
    edge_probabilities = nx.get_edge_attributes(G, 'probability')
    edge_weights = nx.get_edge_attributes(G, 'weight')

    # Kết hợp xác suất và trọng số vào nhãn cạnh
    edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

    # Vẽ đồ thị
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=200, font_size=8, font_weight='bold')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

    plt.title('Uncertain Weighted Directed Graph Visualization')
    plt.show()
    
def average_edge_probability(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0



def calculate_f_beta_v3(G_S, beta, S, T):
    num_edges = G_S.number_of_edges()    
    if len(S) > 0 and len(T) > 0:
        f_beta = (weighted_average_edge_probability(G_S) - beta) * (num_edges / (len(S) * len(T))**0.5)
    else:
        f_beta = 0
    return f_beta
    

def weighted_average_edge_probability(G):
    total_weighted_probability = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())  
    num_edges = G.number_of_edges()
    return total_weighted_probability / num_edges if num_edges > 0 else 0

def surplus_degree_out(G, v, beta):
    return sum(G[v][w]['weight'] *G[v][w]['probability'] - beta for w in G.successors(v) if w in G)

def surplus_degree_in(G, v, beta):
    return sum(G[u][v]['weight'] *G[u][v]['probability'] - beta for u in G.predecessors(v) if u in G)


def initialize_priority_queue_out(G, beta):
    """ Khởi tạo hàng đợi ưu tiên với bậc dư thừa cho mỗi đỉnh """
    priority_queue = []
    for v in G.nodes():
        sd = surplus_degree_out(G, v, beta)
        heapq.heappush(priority_queue, (sd, v))
        
    return priority_queue

def initialize_priority_queue_in(G, beta):
    """ Khởi tạo hàng đợi ưu tiên với bậc dư thừa cho mỗi đỉnh """
    priority_queue = []
    for v in G.nodes():
        sd = surplus_degree_in(G, v, beta)
        heapq.heappush(priority_queue, (sd, v))
        
    return priority_queue


def calculate_edge_density_v3(G):
    """Tính mật độ cạnh kỳ vọng của đồ thị G. τ"""
    num_vertices = len(G.nodes())
    num_possible_edges = num_vertices * (num_vertices - 1) if num_vertices > 1 else 1
    sum_weighted_probabilities = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())
    return sum_weighted_probabilities / num_possible_edges

def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5

def trong_so_trung_binh_canh(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'weight').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0
def update_priority_queue_in(H, beta, neighbors, priority_queue_in):
    for neighbor in neighbors:
        if neighbor in H:
            sd_in = surplus_degree_in(H, neighbor, beta)
            heapq.heappush(priority_queue_in, (sd_in, neighbor))

def update_priority_queue_out(H, beta, neighbors, priority_queue_out):
    for neighbor in neighbors:
        if neighbor in H:
            sd_out = surplus_degree_out(H, neighbor, beta)
            heapq.heappush(priority_queue_out, (sd_out, neighbor))

        

def greedy_average_surplus_degree(G, beta, S, T, c):
    H = G.copy()
    S_temp = S
    T_temp = T
    best_subgraph = H.copy()
    best_f_beta = calculate_f_beta_v3(best_subgraph, beta, S_temp, T_temp)


    num_iterations = len(S_temp) + len(T_temp)  # Tổng số đỉnh ban đầu trong cả hai tập
    with tqdm(total=num_iterations, desc="Processing", unit="node") as pbar:
        while S_temp and T_temp:
            i_min = min(S_temp, key=lambda i: surplus_degree_out(H, i, beta))
            d_S = surplus_degree_out(H, i_min, beta)

            j_min = min(T_temp, key=lambda j: surplus_degree_in(H, j, beta))
            d_T = surplus_degree_in(H, j_min, beta)
        
            
            if c * d_S <= d_T / c:
                # Loại bỏ i_min khỏi S_temp, nhưng giữ lại trong T_temp nếu có
                S_temp.remove(i_min)
                if i_min not in T_temp:  # Nếu i_min không có trong T_temp, loại nó khỏi đồ thị
                    H.remove_node(i_min)
                else:
                    # Nếu i_min vẫn còn trong T_temp, chỉ loại bỏ các cạnh đi ra từ i_min
                    for w in list(H.successors(i_min)):
                        if w in T_temp:
                            H.remove_edge(i_min, w)
            else:
                # Loại bỏ j_min khỏi T_temp, nhưng giữ lại trong S_temp nếu có
                T_temp.remove(j_min)
                if j_min not in S_temp:  # Nếu j_min không có trong S_temp, loại nó khỏi đồ thị
                    H.remove_node(j_min)
                else:
                    # Nếu j_min vẫn còn trong S_temp, chỉ loại bỏ các cạnh đi vào j_min
                    for u in list(H.predecessors(j_min)):
                        if u in S_temp:
                            H.remove_edge(u, j_min)

            current_f_beta = calculate_f_beta_v3(H, beta, S_temp, T_temp)
            
            S_temp = {u for u in S_temp if u in H}
            T_temp = {v for v in T_temp if v in H}
            
            if current_f_beta > best_f_beta:
                best_f_beta = current_f_beta
                best_subgraph = H.copy()  # Lưu bản sao của H với mật độ cao nhất

            
            pbar.update(1)  # Cập nhật tiến trình mỗi khi một đỉnh được loại bỏ
    
    return best_subgraph





In [3]:
random.seed(9)
# Đọc dữ liệu từ tệp TXT
file_path = '/teamspace/uploads/data/1736402.protein.links.full.v12.0.txt.gz'
data = pd.read_csv(file_path, sep=' ')

# Khởi tạo đồ thị có hướng
G = nx.DiGraph()

# Tìm trọng số lớn nhất trong cột 'experiments'
max_experiment_weight = data['experiments_transferred'].max()

# Thêm các cạnh với trọng số và xác suất
for _, row in data.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    experiment_weight = row['experiments_transferred']

    # Tính tỷ lệ trọng số so với trọng số lớn nhất
    if max_experiment_weight > 0:
        normalized_weight = experiment_weight / max_experiment_weight
    else:
        normalized_weight = 0  # Tránh chia cho 0 trong trường hợp không có trọng số nào

    # Giả sử xác suất được tính dựa trên combined_score
    combined_score = row['combined_score']
    probability = combined_score / 1000.0  # Điều chỉnh hệ số nếu cần thiết

    # Định hướng cạnh dựa trên một quy tắc ngẫu nhiên
    if random.choice([True, False]):
        G.add_edge(protein1, protein2, probability=probability, weight=normalized_weight)  
        
S = set()
T = set()
for u, v in G.edges():
    S.add(u)
    T.add(v)
# print("tập S có:",len(S))
# print("tập T có:",len(T))

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)
do_lech_chuan_trong_so = np.std(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Độ lệch chuẩn của của  đồ thị protein579138:", std_deviation)
print("Độ lệch chuẩn trọng số của của  đồ thị protein579138:", do_lech_chuan_trong_so)

print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(G))


Number of nodes: 3085
Number of edges: 314788
Giá trị cạnh trung bình của đồ thị protein579138: 0.28146895688526885
Độ lệch chuẩn của của  đồ thị protein579138: 0.18211671967697324
Độ lệch chuẩn trọng số của của  đồ thị protein579138: 0.14835576350857846
Trọng số cạnh trung bình của đồ thị là: 0.05057740498418036
Mật độ kì cạnh vọng của thuật toán obs  là: 0.0009661932566591212


In [4]:
beta = 0.2 * 0.2
c = len(S) / len(T)
ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta, S, T, c)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
std_weight= np.std(weights)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn xác xuất:", std_deviation)
print("Độ lệch chuẩn trọng số:", std_weight)
print("Trung bình trọng số xác xuất :",weighted_average_edge_probability(ObsEmailEuCore01))

Processing: 100%|█████████▉| 6123/6124 [54:06<00:00,  1.89node/s]  


Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là: 30.439996003485977
Số đỉnh trong đồ thị con của thuật toán obs  là: 106
Số cạnh trong đồ thị con của thuật toán obs  là: 4294
Mật độ kì cạnh vọng của thuật toán obs  là: 0.26648578056942834
Xác xuất cạnh trung bình của thuật toán obs là: 0.8632899394503913
 trọng số cạnh trung bình của thuật toán obs là: 0.750118073359806
Độ lệch chuẩn xác xuất: 0.1934200597702181
Độ lệch chuẩn trọng số: 0.2606746014207703
Trung bình trọng số xác xuất : 0.690728164354387


In [5]:
nx.write_gexf(ObsEmailEuCore01, "Obs_1736402_02.gexf")
